In [2]:
import pandas as pd
import os
import numpy as np
from sklearn.metrics import roc_curve, auc, confusion_matrix
import matplotlib.pyplot as plt

In [3]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import roc_curve, auc, confusion_matrix
import matplotlib.pyplot as plt

import boto3
import sagemaker


In [4]:
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

## Check if data is available at S3

In [4]:
# set prefix, a descriptive name for a directory  
prefix = 'anomaly'

for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    print(obj.key)

anomaly/RandomCutForest-2021-06-22-10-27-23-663/.amazon.manifest
anomaly/RandomCutForest-2021-06-22-10-27-23-663/matrix_0.pbr
anomaly/RandomCutForest-2021-06-22-11-34-41-100/.amazon.manifest
anomaly/RandomCutForest-2021-06-22-11-34-41-100/matrix_0.pbr
anomaly/RandomCutForest-2021-06-23-07-14-52-665/.amazon.manifest
anomaly/RandomCutForest-2021-06-23-07-14-52-665/matrix_0.pbr
anomaly/output/randomcutforest-2021-06-22-10-34-28-100/output/model.tar.gz
anomaly/output/randomcutforest-2021-06-22-10-34-28-100/profiler-output/framework/training_job_end.ts
anomaly/output/randomcutforest-2021-06-22-10-34-28-100/profiler-output/system/incremental/2021062210/1624358220.algo-1.json
anomaly/output/randomcutforest-2021-06-22-10-34-28-100/profiler-output/system/incremental/2021062210/1624358280.algo-1.json
anomaly/output/randomcutforest-2021-06-22-10-34-28-100/profiler-output/system/incremental/2021062210/1624358340.algo-1.json
anomaly/output/randomcutforest-2021-06-22-10-34-28-100/profiler-output/sys

In [5]:
# set locations from variables from data preperation notebook
train_location = 's3://sagemaker-us-east-1-517714493426/anomaly/train.csv'
val_location = 's3://sagemaker-us-east-1-517714493426/anomaly/val.csv'
test_location = 's3://sagemaker-us-east-1-517714493426/anomaly/test.csv'

## Train a Model with Sagemaker built-in function Random Cut Forests

In [15]:
import sagemaker

region = sagemaker.Session().boto_region_name
print("AWS Region: {}".format(region))

role = sagemaker.get_execution_role()
print("RoleArn: {}".format(role))

#https://docs.aws.amazon.com/sagemaker/latest/dg/ecr-us-east-1.html#randomcutforest-us-east-1.title
container=sagemaker.image_uris.retrieve("randomcutforest", region, "us-east-1")
print(container)

rcf = sagemaker.estimator.Estimator(
    image_uri=container,
    role=role,
    instance_count=1,
    sagemaker_session=sagemaker.Session(),
    instance_type="ml.m4.xlarge",
    data_location=f"s3://{bucket}/{prefix}/",
    output_path=f"s3://{bucket}/{prefix}/output"
)

rcf.set_hyperparameters(
    eval_metrics = 'precision_recall_fscore',
    num_samples_per_tree=256,
    num_trees=100,
    
)

train_data = sagemaker.inputs.TrainingInput(s3_data=train_location, content_type='text/csv;label_size=0', distribution='ShardedByS3Key')
val_data = sagemaker.inputs.TrainingInput(s3_data=val_location, content_type='csv', distribution='FullyReplicated')

rcf.fit({'train': train_data, 'validation': val_data}, wait=True)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: us-east-1.


AWS Region: us-east-1
RoleArn: arn:aws:iam::517714493426:role/service-role/AmazonSageMaker-ExecutionRole-20210409T152960
382416733822.dkr.ecr.us-east-1.amazonaws.com/randomcutforest:1


AttributeError: 'Estimator' object has no attribute 'record_set'

In [16]:
from sagemaker import RandomCutForest

prefix = 'anomaly'

# specify general training job information
rcf = RandomCutForest(
    role=role,
    instance_count=1,
    instance_type="ml.m4.xlarge",
    data_location=f"s3://{bucket}/{prefix}/",
    output_path=f"s3://{bucket}/{prefix}/output",
    num_samples_per_tree=512,
    num_trees=50,
    eval_metrics='f1'
)

#https://sagemaker-examples.readthedocs.io/en/latest/introduction_to_amazon_algorithms/random_cut_forest/random_cut_forest.html
data_train = pd.read_pickle('X_train.pkl')
X_val = pd.read_pickle('X_val.pkl')
y_val = pd.read_pickle('y_val.pkl')
data_val = pd.concat([y_val, X_val], axis=1)

# automatically upload the training data to S3 and run the training job
rcf.fit(rcf.record_set(data_train.to_numpy()), rcf.record_set(data_val.to_numpy()))

ValueError: Random Cut Forest uses a fixed mini_batch_size of 1000

In [5]:
from sagemaker import RandomCutForest

prefix = 'anomaly'

# specify general training job information
rcf = RandomCutForest(
    role=role,
    instance_count=1,
    instance_type="ml.m4.xlarge",
    data_location=f"s3://{bucket}/{prefix}/",
    output_path=f"s3://{bucket}/{prefix}/output",
    num_samples_per_tree=512,
    num_trees=50,
    eval_metrics='f1'
)

#https://sagemaker-examples.readthedocs.io/en/latest/introduction_to_amazon_algorithms/random_cut_forest/random_cut_forest.html
data_train = pd.read_pickle('X_train.pkl')

# automatically upload the training data to S3 and run the training job
rcf.fit(rcf.record_set(data_train.to_numpy()))

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2021-06-24 10:55:24 Starting - Starting the training job...
2021-06-24 10:55:26 Starting - Launching requested ML instancesProfilerReport-1624532124: InProgress
......
2021-06-24 10:56:41 Starting - Preparing the instances for training.........
2021-06-24 10:58:21 Downloading - Downloading input data...
2021-06-24 10:58:43 Training - Downloading the training image..Docker entrypoint called with argument(s): train
Running default environment configuration script
[06/24/2021 10:59:09 INFO 139736704735040] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-conf.json: {'num_samples_per_tree': 256, 'num_trees': 100, 'force_dense': 'true', 'eval_metrics': ['accuracy', 'precision_recall_fscore'], 'epochs': 1, 'mini_batch_size': 1000, '_log_level': 'info', '_kvstore': 'dist_async', '_num_kv_servers': 'auto', '_num_gpus': 'auto', '_tuning_objective_metric': '', '_ftp_port': 8999}
[06/24/2021 10:59:09 INFO 139736704735040] Merging with provided


2021-06-24 10:59:23 Uploading - Uploading generated training model
2021-06-24 10:59:23 Completed - Training job completed
[2021-06-24 10:59:11.707] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 1, "duration": 725, "num_examples": 346, "num_bytes": 330604876}
[06/24/2021 10:59:11 INFO 139736704735040] Sampling training data completed.
#metrics {"StartTime": 1624532350.975899, "EndTime": 1624532351.7561133, "Dimensions": {"Algorithm": "RandomCutForest", "Host": "algo-1", "Operation": "training"}, "Metrics": {"epochs": {"sum": 1.0, "count": 1, "min": 1, "max": 1}, "update.time": {"sum": 774.5928764343262, "count": 1, "min": 774.5928764343262, "max": 774.5928764343262}}}

[06/24/2021 10:59:11 INFO 139736704735040] Early stop condition met. Stopping training.
[06/24/2021 10:59:11 INFO 139736704735040] #progress_metric: host=algo-1, completed 100 % epochs
#metrics {"StartTime": 1624532350.9814672, "EndTime": 1624532351.756615, "Dimensions": {"Algorithm

In [ ]:
print(f"Training job name: {rcf.latest_training_job.job_name}")

In [ ]:
rcf_transformer = rcf.transformer(instance_count = 1, instance_type = 'ml.m4.xlarge')

# TODO: Start the transform job. Make sure to specify the content type and the split type of the test data.
rcf_transformer.transform(test_location, content_type='text/csv', split_type='Line')
rcf_transformer.wait()


## Analyzing results and set treshold

In [ ]:
data_dir = 'data'
# copy result on S3 to local notebook instance
!aws s3 cp --recursive $rcf_transformer.output_path $data_dir

In [ ]:
data_dir = 'data'
predictions_raw = pd.read_csv(os.path.join(data_dir, 'test.csv.out'), sep=",", header=None)
print(predictions_raw)

In [ ]:
def get_scores(predictions):
    pred = np.empty(predictions.shape[0])
    
    for i in range(predictions.shape[0]):
        pred[i] = predictions.iloc[i][0].split(":",1)[1][:-1]
        
    return pred

In [ ]:
pred = get_scores(predictions_raw)

In [ ]:
pred_df = pd.DataFrame(pred)
pred_df.describe()

In [ ]:
#distribution
import matplotlib.pyplot as plt
plt.xlabel('Anomaly Score')
pred_df[0].plot.hist(bins=200, range=[0, 2])

In [ ]:
y_test = pd.read_pickle("y_test.pkl")
y_test = pd.DataFrame({'label': y_test, 'score': pred})

In [ ]:
y_test

In [ ]:
y_test.groupby(by='label').describe()

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test['label'].values, y_test['score'].values)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(10,10))
plt.plot(fpr, tpr, lw=1, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='lime', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

## Inference 

In [ ]:
rcf_inference = rcf.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")
print(f"Endpoint name: {rcf_inference.endpoint}")

In [ ]:
# only work vor old sagemaker version
#from sagemaker.predictor import csv_serializer
#
#rcf_inference.content_type = 'text/csv'
#rcf_inference.serializer = csv_serializer

## Data Serialization

In [ ]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

rcf_inference.serializer = CSVSerializer()
rcf_inference.deserializer = JSONDeserializer()

In [ ]:
X_test = pd.read_pickle("X_test.pkl")
y_test = pd.read_pickle("y_test.pkl")

In [ ]:
X_test.values

In [ ]:
#X_test_numpy = X_test.values.to_numpy().reshape(-1, 1)
X_test_numpy = X_test.values.reshape(-1, 1)
#print(X_test_numpy[:6])
results = rcf_inference.predict(
    X_test_numpy, initial_args={"ContentType": "text/csv", "Accept": "application/json"}
)

## Predicion & Anomaly Scores

In [ ]:
results = rcf_inference.predict(X_test_numpy)
scores = [entry["score"] for entry in results["scores"]]

# add scores to taxi data frame and print first few values
y_test_predict = pd.Series(scores, index=X_test.index)
y_test_predict.head()